# Introduction

------------------------------
## Mission - Élaborez le modèle de scoring
------------------------------

Vous êtes Data Scientist au sein d'une société financière, nommée **"Prêt à dépenser"**, qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite **mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité** qu’un client rembourse son crédit, puis classifier la demande en crédit accordé ou refusé. Elle souhaite donc développer un **algorithme de classification** en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.)

**Votre mission :**
* Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.
* Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.
* Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.
* Mettre en œuvre une approche globale MLOps de bout en bout, du tracking des expérimentations à l’analyse en production du data drift.

Michaël, votre manager, vous incite à sélectionner un ou des kernels Kaggle pour vous faciliter l’analyse exploratoire, la préparation des données et le feature engineering nécessaires à l’élaboration du modèle de scoring. 

Voici le mail qu’il vous a envoyé.

*Bonjour,*

*Afin de pouvoir faire évoluer régulièrement le modèle, je souhaite tester la mise en œuvre une démarche de type MLOps d’automatisation et d’industrialisation de la gestion du cycle de vie du modèle.*

*Vous trouverez en pièce jointe **la liste d’outils à utiliser** pour créer une plateforme MLOps qui s’appuie sur des outils Open Source.* 

*Je souhaite que vous puissiez mettre en oeuvre au minimum **les étapes orientées MLOps** suivantes :* 
* *Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations*
* *Lancer l’interface web 'UI MLFlow" d'affichage des résultats du tracking*
* *Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”*
* *Tester le serving MLFlow*
* *Gérer le code avec le logiciel de version Git*
* *Partager le code sur Github pour assurer une intégration continue*
* *Utiliser Github Actions pour le déploiement continu et automatisé du code de l’API sur le cloud*
* *Concevoir des tests unitaires avec Pytest (ou Unittest) et les exécuter de manière automatisée lors du build réalisé par Github Actions*
 
*J’ai également rassemblé des conseils pour vous aider à vous lancer dans ce projet !*

*Concernant **l’élaboration du modèle** soyez vigilant sur deux points spécifiques au contexte métier :* 
* *Le déséquilibre entre le nombre de bons et de moins bons clients doit être pris en compte pour élaborer un modèle pertinent, avec une méthode au choix*
* *Le déséquilibre du coût métier entre un faux négatif (FN - mauvais client prédit bon client : donc crédit accordé et perte en capital) et un faux positif (FP - bon client prédit mauvais : donc refus crédit et manque à gagner en marge)*
    * *Vous pourrez supposer, par exemple, que le coût d’un FN est dix fois supérieur au coût d’un FP*
    * *Vous créerez un score “métier” (minimisation du coût d’erreur de prédiction des FN et FP) pour comparer les modèles, afin de choisir le meilleur modèle et ses meilleurs hyperparamètres. Attention cette minimisation du coût métier doit passer par l’optimisation du seuil qui détermine, à partir d’une probabilité, la classe 0 ou 1 (un “predict” suppose un seuil à 0.5 qui n’est pas forcément l’optimum)*
    * *En parallèle, maintenez pour comparaison et contrôle des mesures plus techniques, telles que l’AUC et l’accuracy*
 
*D’autre part je souhaite que vous mettiez en œuvre une démarche d’élaboration des modèles avec **Cross-Validation et optimisation des hyperparamètres, via GridsearchCV ou équivalent.***

*Un dernier conseil : si vous obtenez des scores supérieurs au 1er du challenge Kaggle (AUC > 0.82), posez-vous la question si vous n’avez pas de l’overfitting dans votre modèle !*

*Vous exposerez votre **modèle de prédiction sous forme d’une API** qui permet de calculer la probabilité de défaut du client, ainsi que sa classe (accepté ou refusé) en fonction du seuil optimisé d’un point de vue métier.*

***Le déploiement de l’API** sera réalisée sur une plateforme Cloud, de préférence une solution gratuite.*

*Je vous propose d’utiliser un Notebook ou une application Streamlit pour réaliser en local  l’**interface de test de l’API**.*

*Bon courage !*

*Mickael*

# Import des librairies

In [1]:
## Global 
import os
import shutil
import time
import pandas as pd
import numpy as np
import missingno as msno

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.metrics import confusion_matrix

# MLFlow
import mlflow

# Optuna
import optuna

In [2]:
# Initialize sns
sns.set()

# Lecture du dataset

In [3]:
# Define the path to data
path = "./data/"
# List files in the data directory
file_list = os.listdir(path)
# Create an empty dict to store the file name as key and a DataFrame as value
df_dict = {}
# Create a list to store file names
filenames_list = []
# Go through the list and get the name of the file without extension
for file in file_list:
    # Get the name of the file and its extension
    name, extension = os.path.splitext(file)
    # Append the name of the file to the dedicated list
    filenames_list.append(name)
    # Exclude "HomeCredit_columns_description.csv" which is not usefull for our purpose
    if file != "HomeCredit_columns_description.csv":
        # Append the dataframe read to the list
        df_dict[name] = pd.read_csv(path+file)

# Préparation des données

Pour la préparation des données nous nous appuierons sur le kernel Kaggle suivant : <a href="https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script">LightGBM with Simple Features</a>

Au préalable, définissons une fonction nous permettant d'effectuer un OneHotEnconding sur les variables catégorielles.

Celle-ci nous retournera le dataframe modifié après l'encoding et une liste des nouvelles colonnes créées.

In [4]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encode(data, nan_as_category=True):
    # Store the name of original columns
    original_columns = list(data.columns)
    # Retrieve categorical columns
    categorical_columns = [col for col in data.columns if data[col].dtype == "object"]
    # Proceed with OneHotEncoding
    data = pd.get_dummies(data, columns= categorical_columns, dummy_na= nan_as_category)
    # List new columns
    new_columns = [c for c in data.columns if c not in original_columns]
    # Return the dataframe and the list of new columns
    return data, new_columns

Premièrement intéressons nous aux dataframes "application"

In [5]:
# Merge train and test dataframes
df = pd.concat([df_dict["application_train"], df_dict["application_test"]]).reset_index(drop=True)

# Remove XNA gender
df = df[df["CODE_GENDER"] != "XNA"]

# Binary encode two state categorical features
for binary_features in ["NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY"]:
    df[binary_features], uniques = pd.factorize(df[binary_features])
    
# OneHot Encode categorical features
df, applications_cat_cols = one_hot_encode(df, False)

# Replace 365243 by NaN in DAYS_EMPLOYED column
df.loc[df["DAYS_EMPLOYED"]==365243, ["DAYS_EMPLOYED"]] = np.nan

# Generate new features by calculating percentages
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

A présent, intéressons nous aux dataframes "bureau" et "bureau_balance"

In [6]:
# OneHot Encode categorical features
df_dict["bureau"], bureau_cat_cols = one_hot_encode(df_dict["bureau"], True)

# OneHot Encode categorical features
df_dict["bureau_balance"], bureau_balance_cat_cols = one_hot_encode(df_dict["bureau_balance"], True)

# Aggregation of bureau_balance dataframe on SK_ID_BUREAU
aggregations = {
    "MONTHS_BALANCE": ["min", "max", "size"], 
}
for col in bureau_balance_cat_cols:
    aggregations[col] = ["mean"]
# Groupby "SK_ID_BUREAU"
bb_agg = df_dict["bureau_balance"].groupby("SK_ID_BUREAU").agg(aggregations)
# Rename columns
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

# Merge bureau and bureau_balance dataframes
bureau = df_dict["bureau"].join(bb_agg, how="left", on="SK_ID_BUREAU")
# Drop SK_ID_BUREAU column, which is a reference between bureau and bureau_balance dataframes
bureau.drop(["SK_ID_BUREAU"], axis=1, inplace= True)

# Aggregation of bureau dataframe
# Build numeric features in bureau dataframe
num_aggregations = {
    "DAYS_CREDIT": ["min", "max", "mean", "var"],
    "DAYS_CREDIT_ENDDATE": ["min", "max", "mean"],
    "DAYS_CREDIT_UPDATE": ["mean"],
    "CREDIT_DAY_OVERDUE": ["max", "mean"],
    "AMT_CREDIT_MAX_OVERDUE": ["mean"],
    "AMT_CREDIT_SUM": ["max", "mean", "sum"],
    "AMT_CREDIT_SUM_DEBT": ["max", "mean", "sum"],
    "AMT_CREDIT_SUM_OVERDUE": ["mean"],
    "AMT_CREDIT_SUM_LIMIT": ["mean", "sum"],
    "AMT_ANNUITY": ["max", "mean"],
    "CNT_CREDIT_PROLONG": ["sum"],
    "MONTHS_BALANCE_MIN": ["min"],
    "MONTHS_BALANCE_MAX": ["max"],
    "MONTHS_BALANCE_SIZE": ["mean", "sum"],
}
# Build categorical features
cat_aggregations = {}
for cat in bureau_cat_cols: cat_aggregations[cat] = ["mean"]
for cat in bureau_balance_cat_cols: cat_aggregations[cat + "_MEAN"] = ["mean"]
# Groupby "SK_ID_CURR"
bureau_agg = bureau.groupby("SK_ID_CURR").agg({**num_aggregations, **cat_aggregations})
# Rename columns
bureau_agg.columns = pd.Index(["BURO_" + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

# Generate features for active credits
# Get all active credits
active = bureau[bureau["CREDIT_ACTIVE_Active"] == 1]
# Groupby "SK_ID_CURR" with the same aggregations params for numerical features only
active_agg = active.groupby("SK_ID_CURR").agg(num_aggregations)
# Rename columns
active_agg.columns = pd.Index(["ACTIVE_" + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
# Join bureau_agg and active_agg dataframes
bureau_agg = bureau_agg.join(active_agg, how="left", on="SK_ID_CURR")

# Generate features for closed credits
# Get all closed credits
closed = bureau[bureau["CREDIT_ACTIVE_Closed"] == 1]
# Groupby "SK_ID_CURR" with the same aggregations params for numerical features only
closed_agg = closed.groupby("SK_ID_CURR").agg(num_aggregations)
# Rename columns
closed_agg.columns = pd.Index(["CLOSED_" + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
# Join bureau_agg and active_agg dataframes
bureau_agg = bureau_agg.join(closed_agg, how="left", on="SK_ID_CURR")

# Join df and new bureau_agg dataframes
df = df.join(bureau_agg, how="left", on="SK_ID_CURR")

Passons au dataframe "previous_application"

In [7]:
# OneHot Encode categorical features
df_dict["previous_application"], prev_app_cat_cols = one_hot_encode(df_dict["previous_application"], True)

# Replace 365243 days by NaN 
df_dict["previous_application"].loc[
    df_dict["previous_application"]["DAYS_FIRST_DRAWING"] == 365243, ["DAYS_FIRST_DRAWING"] 
] = np.nan
df_dict["previous_application"].loc[
    df_dict["previous_application"]["DAYS_FIRST_DUE"] == 365243, ["DAYS_FIRST_DUE"] 
] = np.nan
df_dict["previous_application"].loc[
    df_dict["previous_application"]["DAYS_LAST_DUE_1ST_VERSION"] == 365243, ["DAYS_LAST_DUE_1ST_VERSION"] 
] = np.nan
df_dict["previous_application"].loc[
    df_dict["previous_application"]["DAYS_LAST_DUE"] == 365243, ["DAYS_LAST_DUE"] 
] = np.nan
df_dict["previous_application"].loc[
    df_dict["previous_application"]["DAYS_TERMINATION"] == 365243, ["DAYS_TERMINATION"] 
] = np.nan

# Add feature : value ask / value received percentage
df_dict["previous_application"]['APP_CREDIT_PERC'] = \
    df_dict["previous_application"]['AMT_APPLICATION'] / df_dict["previous_application"]['AMT_CREDIT']

# Aggregation of previous_application dataframe
# Previous applications numeric features
num_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
}
# Previous applications categorical features
cat_aggregations = {}
for cat in prev_app_cat_cols:
    cat_aggregations[cat] = ['mean']
# Groupby "SK_ID_CURR"
prev_agg = df_dict["previous_application"].groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
# Rename columns
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

# Generate features for approved applications
# Get all approved applications
approved = df_dict["previous_application"][df_dict["previous_application"]["NAME_CONTRACT_STATUS_Approved"] == 1]
# Groupby "SK_ID_CURR" with the same aggregations params for numerical features only
approved_agg = approved.groupby("SK_ID_CURR").agg(num_aggregations)
# Rename columns
approved_agg.columns = pd.Index(["APPROVED_" + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
# Join prev_agg and approved_agg dataframes
prev_agg = prev_agg.join(approved_agg, how="left", on="SK_ID_CURR")

# Generate features for refused applications
# Get all refused applications
refused = df_dict["previous_application"][df_dict["previous_application"]["NAME_CONTRACT_STATUS_Refused"] == 1]
# Groupby "SK_ID_CURR" with the same aggregations params for numerical features only
refused_agg = refused.groupby("SK_ID_CURR").agg(num_aggregations)
# Rename columns
refused_agg.columns = pd.Index(["REFUSED_" + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
# Join prev_agg and refused_agg dataframes
prev_agg = prev_agg.join(refused_agg, how="left", on="SK_ID_CURR")

# Join df and new bureau_agg dataframes
df = df.join(prev_agg, how="left", on="SK_ID_CURR")

Au tour du dataframe "POS_CASH_balance"

In [8]:
# OneHot Encode categorical features
df_dict["POS_CASH_balance"], pos_cat_cols = one_hot_encode(df_dict["POS_CASH_balance"], True)

# Aggregation of POS_CASH_balance dataframe
# POS_CASH_balance numeric features
aggregations = {
    "MONTHS_BALANCE": ["max", "mean", "size"],
    "SK_DPD": ["max", "mean"],
    "SK_DPD_DEF": ["max", "mean"],
}
# POS_CASH_balance categorical features
for cat in pos_cat_cols:
    aggregations[cat] = ["mean"]
# Groupby "SK_ID_CURR"
pos_agg = df_dict["POS_CASH_balance"].groupby("SK_ID_CURR").agg(aggregations)
# Rename columns
pos_agg.columns = pd.Index(["POS_" + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
# Count POS CASH accounts
pos_agg["POS_COUNT"] = df_dict["POS_CASH_balance"].groupby("SK_ID_CURR").size

# Join df and new pos_agg dataframes
df = df.join(pos_agg, how="left", on="SK_ID_CURR")

Puis du dataframe "installments_payments"

In [9]:
# OneHot Encode categorical features
df_dict["installments_payments"], install_cat_cols = one_hot_encode(df_dict["installments_payments"], True)

# Percentage and difference paid in each installment (amount paid and installment value)
df_dict["installments_payments"]["PAYMENT_PERC"] = \
    df_dict["installments_payments"]["AMT_PAYMENT"] / df_dict["installments_payments"]["AMT_INSTALMENT"]
df_dict["installments_payments"]["PAYMENT_DIFF"] = \
    df_dict["installments_payments"]["AMT_INSTALMENT"] - df_dict["installments_payments"]["AMT_PAYMENT"]
# Days past due and days before due (no negative values)
df_dict["installments_payments"]["DPD"] = \
    df_dict["installments_payments"]["DAYS_ENTRY_PAYMENT"] - df_dict["installments_payments"]["DAYS_INSTALMENT"]
df_dict["installments_payments"]["DBD"] = \
    df_dict["installments_payments"]["DAYS_INSTALMENT"] - df_dict["installments_payments"]["DAYS_ENTRY_PAYMENT"]
df_dict["installments_payments"]["DPD"] = \
    df_dict["installments_payments"]["DPD"].apply(lambda x: x if x > 0 else 0)
df_dict["installments_payments"]["DBD"] = \
    df_dict["installments_payments"]["DBD"].apply(lambda x: x if x > 0 else 0)

# Aggregation of installments_payments dataframe
# installments_payments numeric features
aggregations = {
    "NUM_INSTALMENT_VERSION": ["nunique"],
    "DPD": ["max", "mean", "sum"],
    "DBD": ["max", "mean", "sum"],
    "PAYMENT_PERC": ["max", "mean", "sum", "var"],
    "PAYMENT_DIFF": ["max", "mean", "sum", "var"],
    "AMT_INSTALMENT": ["max", "mean", "sum"],
    "AMT_PAYMENT": ["min", "max", "mean", "sum"],
    "DAYS_ENTRY_PAYMENT": ["max", "mean", "sum"],
}
# installments_payments categorical features
for cat in install_cat_cols:
    aggregations[cat] = ["mean"]
# Groupby "SK_ID_CURR"
ins_agg = df_dict["installments_payments"].groupby("SK_ID_CURR").agg(aggregations)
# Rename columns
ins_agg.columns = pd.Index(["INSTAL_" + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
# Count POS CASH accounts
ins_agg["INSTAL_COUNT"] = df_dict["installments_payments"].groupby("SK_ID_CURR").size

# Join df and new ins_agg dataframes
df = df.join(ins_agg, how="left", on="SK_ID_CURR")

Et enfin du dataframe "credit_card_balance"

In [10]:
# OneHot Encode categorical features
df_dict["credit_card_balance"], credit_cat_cols = one_hot_encode(df_dict["credit_card_balance"], True)

# General aggregations
df_dict["credit_card_balance"].drop(["SK_ID_PREV"], axis= 1, inplace = True)
cc_agg = df_dict["credit_card_balance"].groupby("SK_ID_CURR").agg(["min", "max", "mean", "sum", "var"])
cc_agg.columns = pd.Index(["CC_" + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
# Count credit card lines
cc_agg["CC_COUNT"] = df_dict["credit_card_balance"].groupby("SK_ID_CURR").size()

# Join df and new cc_agg dataframes
df = df.join(cc_agg, how="left", on="SK_ID_CURR")

Séparons à présent nos données en jeux d'entrainement et de test

In [11]:
# Divide in training/validation and test data
train_df = pd.DataFrame(df[df['TARGET'].notnull()])
test_df = pd.DataFrame(df[df['TARGET'].isnull()])

# Vérification des correlations

Vérifions qu'après la création de nos features, nous n'avons pas de correlations trop fortes avec notre cible, auquel cas ces variables seraient supprimées pour éviter le data leakage.

Tout d'abord, nous vérifierons les corrrelations de Pearson, puis les correlations de Spearman.

In [12]:
# Compute pearson correlation matrix
pearson = train_df.corr(method="pearson", numeric_only=True)

In [21]:
# Compute spearman correlation matrix
spearman = train_df.corr(method="spearman", numeric_only=True)

In [53]:
# Look for correlations or anti correlations with the target of more than 70%
pearson["TARGET"].loc[(pearson["TARGET"] > 0.7) | (pearson["TARGET"] < -0.7)]

TARGET    1.0
Name: TARGET, dtype: float64

In [52]:
# Look for correlations or anti correlations with the target of more than 70%
spearman["TARGET"].loc[(spearman["TARGET"] > 0.7) | (spearman["TARGET"] < -0.7)]

TARGET    1.0
Name: TARGET, dtype: float64

Aucune correlation ne semble trop importante avec notre cible !

# Fonction de scoring

Afin de nous adapter au mieux à la problématique métier, nous allons définir une fonction afin de calculer un "score métier".

En effet, il existe un déséquilibre du coût métier entre un faux négatif (un mauvais client prédit comme bon client : donc crédit accordé et perte de capital) et un faux positif (bon client prédit mauvais : refus de crédit et donc manque à gagner en marge). On suppose que le coût d'un faux négatif est 10 fois supérieur à celui d'un faux positif.

Pour cela, nous calculerons un "gain" que fera la société comme suit :
* les vrais positifs rapportent de l'argent et sont donc pondérés à 1
* les faux positifs font perdre de l'argent mais moins que les faux négatifs et sont donc pondérés à -0.01
* les faux négatifs font également perdre de l'argent mais plus que les faux négatifs et sont donc pondérés à -0.1
* les vrais négatifs ne rapportent pas d'argent et n'en font pas perdre non plus donc sont ignorés

Note : les coéfficients de pondération seront éventuellemnt à modifier selon les observations effectuées sur le comportement lors de l'entrainement des modèles.

In [69]:
def compute_business_score(true_vals, pred_vals, fn_coeff=-0.1, fp_coeff=-0.01):
    """
    Compute a business score to evaluate our models
    Goal : penalize false negative compared to false positive
    --------------------
    Arguments :
        true_vals : array-like : correct values
        pred_vals : array-like : predicted values
        fn_coeff : optionnal, float, between 0 and 1 : coefficient to apply to false negative values
        fp_coeff : optionnal, float, between 0 and 1 : coefficient to apply to false positive values
    --------------------
    """
    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(true_vals, pred_vals).ravel()
    
    score = tp + fp_coeff*fp + fn_coeff*fn
    return score

Testons notre fonction avec plusieurs listes de valeurs :
* valeurs vraies : [0, 1, 1]
* introduction d'un faux positif : [1, 1, 1]
* introduction d'un faux négatif : [0, 1, 0]

In [82]:
# Define list of values
correct_values = [0, 1, 1]
false_positive_values = [1, 1, 1]
false_negative_values = [0, 1, 0]

# Compute scores
perfect_score = compute_business_score(correct_values, correct_values)
fp_score = compute_business_score(correct_values, false_positive_values)
fn_score = compute_business_score(correct_values, false_negative_values)

In [83]:
perfect_score

2.0

In [80]:
fp_score

1.99

In [81]:
fn_score

0.9

On constate bien que notre score diminue fortement en présence d'un faux négatif comme attendu, à l'inverse du score avec un faux positif qui diminue nettement moins.